Sınıflandırma problemlerindeki asıl amacımız gelecek verinin hangi sınıfta yer alacağına karar vermektir. Bu amaç doğrultusunda kullandığımız Destek Vektör Makineleri (SVM), düzlem üzerindeki noktaların bir doğru veya hiper düzlem ile ayrıştırılmasını ve sınıflandırılmasını sağlar. 

SVM'ler, özellikle karmaşık ancak küçük veya orta büyüklükteki veri setlerinin sınıflandırılması için çok uygundur. 
SVM sonucunda bir olasılık değeri elde edilmez, direk verinin hangi sınıfa ait olduğu bilgisi elde edilir. 
SVM'ler Linear SVM classification ve Nonlinear SVM classification şeklinde düşünülür. Veri setinin doğrusal olarak ayrılabilmesi durumu Linear SVM Classification'dur. Veri setinin doğrusal olarak ayrılamadığı durum ise Nonlinear SVM classification'dur.

SVM çalışma mantığı şu şekildedir: Her iki sınıftanda birbirlerine doğru en uçtaki noktalar seçilir. Bu noktalar 'support vectors' olarak adlandırılır. Bu seçilen noktalar arasına düz çizgi (Linear SVM classification) çekilir. Çizgi iki sınıfı ayırır. Belirlediğimiz çizgi konumunu duruma göre sağa veya sola çekebiliriz. Bu çizgi, bir SVM sınıflandırıcısının karar sınırını temsil eder. Ve çizgi, en yakın noktalardan (bir başka deyişle eğitim örneklerinden) mümkün olduğunca uzak durur. Seçilen noktalar arasında kalan bölgeye 'margin' denir. Margin ne kadar geniş ise sınıflar o kadar iyi ayrıştırılır. 
Şunuda söylemeliyim ki, SVM'ler özellik ölçeklerine duyarlıdır ve özellik ölçeklendirme (örneğin, Scikit-Learn'in StandardScaler'i) ile karar sınırı çok daha iyi görünür.

Veri setinin doğrusal olarak ayrılabildiği durumda bu bahsettiklerim gayet uygulanabilir. Fakat doğrusal olmayan (nonlinear) veri setlerini işlemeye yönelik bir yaklaşım, polinom özellikleri gibi daha fazla özellik eklemektir ve bazı durumlarda bu, doğrusal olarak ayrılabilir bir veri kümesiyle sonuçlanabilir. Burada bir matematik tekniği olan 'Kernel Trick' kavramıyla çok sık karşılaşacaksınız. Elimizdeki koordinatları belirli Kernel Fonksiyonları ile çarparak çok daha anlamlı hale getirebiliyoruz. Çok yüksek dereceli polinomlarda bile, aslında eklemeye gerek kalmadan birçok polinom özelliği eklemişsiniz gibi aynı sonucu elde etmeyi mümkün kılar. Şimdi biraz hayal gücümüzü çalıştıralım. Bütün noktaların yan yana aynı hizzada sıralı bir şekilde olduğunu düşünün. Bu durumda veri setini doğrusal olarak ayıramayız. Eğer tüm özelliklerin (noktaların) karelerini alırsak, noktaların yeni konumları mükemmel bir şekilde lineer olarak ayrılabilir.

Formülü inceleyelim:
w^T . x + b
Burada, w; ağırlık vektörü , x; girdi vektörü , b; sapmadır. Yeni bir değer için elde edilen sonuç 0'dan küçük ise, karar çizgisinin altında kalan sınıfa daha yakın olacaktır. Sonuç 0'a eşit veya büyükse, karar çizgisinin üst tarafında kalan alana daha yakın olacaktır.
Bu bahsedilenler margin bölgesinde bir örnek (nokta) olmadığı durumunda rahatlıkla uygulanabilir. Tabi ki her zaman her şey bu kadar iyi olmayabilir. Eğer örnekler margin bölgesinde ise buna 'soft margin' denir. Soft margin'de outlier (aykırı nokta yani sınıfından uzaktaki nokta) devre dışı bırakılır. Hard Margin, veriler doğrusal olarak ayrılabiliyorsa çalışır ve aykırı değerlere karşı oldukça duyarlıdır. Bu yüzden bazı durumlarda Soft Margin’i tercih etmemiz gerekebilir. 
Bu dengeyi SVM içerisindeki C hiperparametresi ile kontrol edebiliriz. C ne kadar büyükse Margin o kadar dardır. Ayrıca model overfit olursa C’yi azaltmamız gerekir.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

In [2]:
#iris veri setini çağıralım.
iris = datasets.load_iris()
X = iris["data"][:, (2, 3)] # petal length, petal width
y = (iris["target"] == 2).astype(np.float64) # Iris-Virginica

In [3]:
"""
Pipeline yapısı içerisine modelin özelliklerini yerleştiririz. yapı içerisinde bulunan StandardScaler() ile özellikleri 
ölçeklendirdiğimize dikkat edelim. C değeri 1 ve LinearSVC sınıfını kullanarak Iris-Virginica çiçeklerini algılamak için 
doğrusal(linear) bir SVM modelini eğitelim:
"""
svm_clf = Pipeline([
("scaler", StandardScaler()),
("linear_svc", LinearSVC(C=1, loss="hinge")),
])
svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_svc', LinearSVC(C=1, loss='hinge'))])

In [4]:
predict = svm_clf.predict(X)

In [5]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y, predict)

array([[98,  2],
       [ 4, 46]], dtype=int64)

Sınıflandırıcımızın performansını değerlendirmek amacıyla Confusion Matrix kullandık. 
Output:
array([[98,  2],
       [ 4, 46]], dtype=int64)
Çıktıda görüldüğü üzere 2+4=6 yanlış tahmin ve 98+46=144 doğru tahmin elde edilmiştir.

Nonlinear SVM Classification'dan yukarıda bahsettik. Şimdi de buna dair bir uygulama yapalım. make_moons() fonksiyonunu kullanarak veri setini çağıralım. Bu veri seti, veri noktalarının iki yarım daire şeklinde şekillendirildiği ikili sınıflandırma için bir oyuncak veri setidir.

In [6]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

In [7]:
polynomial_svm_clf = Pipeline([
("poly_features", PolynomialFeatures(degree=3)),
("scaler", StandardScaler()),
("svm_clf", LinearSVC(C=10, loss="hinge"))
])
polynomial_svm_clf.fit(X, y)

Pipeline(steps=[('poly_features', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('svm_clf', LinearSVC(C=10, loss='hinge'))])

In [8]:
predict2 = polynomial_svm_clf.predict(X)

In [9]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y, predict2)

array([[97,  3],
       [ 3, 47]], dtype=int64)

Confusion Matrix çıktısı:
array([[97,  3],
       [ 3, 47]], dtype=int64)
şeklindedir. 3+3=6 yanlış tahmin ve 97+47=144 doğru tahmin elde edilmiştir.

Birde yukarıda bahsettiğim kernel trick'e dair bir uygulama yapalım:

In [10]:
from sklearn.svm import SVC
poly_kernel_svm_clf = Pipeline([
("scaler", StandardScaler()),
("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
])
poly_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=5, coef0=1, kernel='poly'))])

coef0 hiperparametresi, modelin düşük dereceli polinomlara karşı yüksek dereceli polinomlardan ne kadar etkilendiğini kontrol eder.

In [11]:
predict3 = poly_kernel_svm_clf.predict(X)

In [12]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y, predict3)

array([[97,  3],
       [ 3, 47]], dtype=int64)

Output: 
array([[97,  3],
       [ 3, 47]], dtype=int64)
şeklindedir. 3+3=6 yanlış tahmin ve 97+47=144 doğru tahmin elde edilmiştir.
ln [7] kod bloğu ile aynı sonucu verdiğine dikkat edin :)